In [1]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
import torch.optim as optim
from torchvision.transforms import transforms
from torchvision.utils import save_image

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from DataSet import WaterTemperatureDataSet

import pandas as pd
import datetime

In [2]:
folder = "DataSets" 
WTA1_Dataset = pd.read_csv(folder + "\WTA1_DataSet.csv")

WTA1_Dataset.drop(["location"], axis=1, inplace=True)

In [27]:
Data = WaterTemperatureDataSet(WTA1_Dataset, target_cols=["Water_Avg"], numerical_cols=['Air_Min', 'Air_Max', "Air_Avg"], 
                                categorical_cols=[""], discriminator_col="TimeSeries")

train_iter, test_iter = Data.get_loaders(batch_size=365)

In [28]:
iterator = train_iter._get_iterator()

In [29]:
tensor = iterator.next()
print(tensor[0].size(), tensor[1].size())

torch.Size([365, 3]) torch.Size([365, 1])


In [30]:
len(train_iter)

85

In [31]:
from LSTMs import WaterLSTM
from Loss.dilate import dilate_loss 

In [46]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def trainLSTM(neuralNetwork, train_iter, learning_rate, epochs, show_training=True, print_every=5): 
    device = get_default_device()

    neuralNetwork.train()
    neuralNetwork.to(device)

    optimizer = torch.optim.Adam(neuralNetwork.parameters(), lr=learning_rate)

    total_loss, shape_loss, temporal_loss = [], [], []
    loss_function = nn.MSELoss()

    for epoch in range(epochs):
        for series, target in train_iter: 
            series.to(device)
            target.to(device)

            predictions = neuralNetwork(series)

            loss, loss_shape, loss_temporal = torch.tensor(0), torch.tensor(0), torch.tensor(0)
            loss, loss_shape, loss_temporal = loss_function(predictions, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss.append(loss)
        shape_loss.append(loss_shape)
        temporal_loss.append(loss_temporal)

        if show_training:
            if epoch % print_every == 0: 
                print('epoch ', epoch, ' loss ', loss, ' loss shape ', loss_shape, ' loss temporal ', loss_temporal.item())


    epochs = np.arange(0, epochs)
    plt.plot(epochs, total_loss, c='blue')
    plt.plot(epochs, shape_loss, c='green')
    plt.plot(epochs, temporal_loss, c='red')
    plt.xlabel("epochs")
    plt.ylabel("Losses from DILATE Loss")
    plt.plot()

In [47]:
class Simple(nn.Module): 
    def __init__(self, hidden_layers=178):
        super(Simple, self).__init__()
        self.hidden_layers = hidden_layers
        self.l1 = nn.Linear(3, 128)
        self.l2 = nn.Linear(128, 128)
        self.l3 = nn.Linear(128, 365)

    def forward(self, x): 
        x = torch.tanh(self.l1(x))
        x = torch.sigmoid(self.l2(x))
        output = self.l3(x)
        return output

In [48]:
model = Simple()

trainLSTM(model, train_iter, learning_rate=0.1, epochs=1000, print_every=10)

c:\Users\guill\miniconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([365, 1])) that is different to the input size (torch.Size([365, 365])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


TypeError: iteration over a 0-d tensor